In [1]:
import pandas as pd
import numpy as np
import datetime

In [46]:
budget_23 = pd.concat(
    pd.read_excel("TALOUSARVIO 23.xlsx", sheet_name=None, header=[ 10, 11]),
    ignore_index=True,
)
if len(budget_23.columns) != 31:
    raise Exception("Sheets don't follow the same pattern")
# Dropping columns we don't need
budget_23 = budget_23.drop(
    budget_23.columns[[3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]],
    axis=1,
)
# replace yes and no in the housing column
budget_23.iloc[:, 2] = budget_23.iloc[:, 2].map({"K": True, "E": False})

# CHANGE THIS HERE TO ONLY GET PROJECTS AND NOT MISS ANY PROJECT
budget_23 = budget_23[budget_23[budget_23.columns[15]].notna() | budget_23[budget_23.columns[1]].notna()| budget_23[budget_23.columns[2]].notna() | budget_23[budget_23.columns[14]].notna()]
budget_23 = budget_23.replace({np.nan: None, "?": None})


# currYear = datetime.date.today().year
# financeShift = list(budget_23.columns.get_level_values(0))[4] - currYear - 1
# if financeShift != 0:
#     finance_df = budget_23.iloc[:, 4:14]
#     budget_23.iloc[:, 4:14] = finance_df.shift(financeShift, axis=1)

# projects with Ids available
budget_23_only_Ids=budget_23[budget_23[budget_23.columns[15]].apply(type)==int]
# projects with no Ids available
budget_23_no_Ids = budget_23[budget_23[budget_23.columns[15]].apply(type)!=int]



budget_23_only_Ids

Unnamed: 0_level_0 Unnamed: 1_level_0  \
                                    Unnamed: 0_level_1 Unnamed: 1_level_1   
16       Siltasaarenkatu (Toinen linja - Kolmas linja)                 K1   
19   Hakaniementori + (Länsipää Miina Sillanpään katu)               K5.1   
20             Siltasaarenkatu (Hakaniemenr - Hämeent)               K5.1   
21          Signe Branderin terassi (itä+länsi) + kuja               K5.1   
22                                  Näkinkulku (pinta)               K5.1   
..                                                 ...                ...   
481                                    Vaskipellonkuja                 K4   
483                                       Vankkurikuja                 K4   
484                                           Iespolku                 K4   
487                                   Vaskipellonpolku                 K4   
489                                        Kesantokuja                 K4   

    asuntotuotan. ennuste   2023   2024   2025   2026   2027   2028   2029  \
       Kyllä / Ei  1000 € 1000 € 1000 € 1000 € 1000 € 1000 € 1000 € 1000 €   
16           True  1000.0   None   None   None   None   None   None   None   
19          False  3000.0   None   None   None   None   None   None   None   
20           True   500.0   None   None   None   None   None   None   None   
21           True  3500.0   None   None   None   None   None   None   None   
22           True   300.0   None   None   None   None   None   None   None   
..            ...     ...    ...    ...    ...    ...    ...    ...    ...   
481          None    None   None   None   None   None   None   None   None   
483          None    None   None   None   None   None   None   None   None   
484          None    None   None   None   None   None   None   None   None   
487          None    None   None   None   None   None   None   None   None   
489          None    None   None   None   None   None   None   None   None   

      2030   2031   2032                                                     \
    1000 € 1000 € 1000 €                                           1000 €.1   
16    None   None   None  . 23 0,3 milj. Kivityös 24 0,5 milj.  TARKISTA...   
19    None   None   None    Suunnittelun aloitus vasta 25 ja toteutus 26+27   
20    None   None   None                                               None   
21    None   None   None                                               None   
22    None   None   None                                               None   
..     ...    ...    ...                                                ...   
481   None   None   None                                               None   
483   None   None   None                                               None   
484   None   None   None                                               None   
487   None   None   None                                               None   
489   None   None   None                                               None   

       HKR / PW  
    HANKENUMERO  
16         2637  
19          191  
20          839  
21         3174  
22         3132  
..          ...  
481        1507  
483        1509  
484        1510  
487        1503  
489        1515  

[186 rows x 16 columns]

In [57]:
plan_23 = pd.concat(
    pd.read_excel("TOIMINTASUUNNITELMA 23.xlsx", sheet_name=None, header=[0, 1]),
    ignore_index=True,
)
if len(plan_23.columns) != 28:
    raise Exception("Sheets don't follow the same pattern")

plan_23 = plan_23.drop(
    plan_23.columns[
        [5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
    ],
    axis=1,
)

# Replacing all common " with related network and project Ids
plan_23.iloc[:, 1] = plan_23.iloc[:, 1].dropna().replace(["\""], method="ffill").reindex(plan_23.iloc[:, 1].index)
plan_23.iloc[:, 2] = plan_23.iloc[:, 2].dropna().replace(["\""], method="ffill").reindex(plan_23.iloc[:, 2].index)

# Dropping all non project rows
plan_23 = plan_23[plan_23[plan_23.columns[1]].notna() | plan_23[plan_23.columns[2]].notna() | plan_23[plan_23.columns[3]].notna()| plan_23[plan_23.columns[4]].notna() | plan_23[plan_23.columns[7]].notna()]
plan_23 = plan_23.replace({np.nan: None, "?": None})

# plan_23.groupby(plan_23.columns[7]).aggregate({plan_23.columns[1]: lambda x: list(x)})




In [58]:
# merging budget projects with plan projects on same PW id
plan_budget_23_merged = pd.merge(budget_23_only_Ids, plan_23, left_on=[budget_23_only_Ids.columns[15]], right_on=[plan_23.columns[7]], how='inner')
# Fixing data discrepancy between similar columns
plan_budget_23_merged.iloc[:,4]=plan_budget_23_merged.iloc[:,21]
plan_budget_23_merged.iloc[:,0]=plan_budget_23_merged.iloc[:,16]
plan_budget_23_merged.iloc[:,14]=plan_budget_23_merged.iloc[:,22]
# Dropping duplicated columns
plan_budget_23_merged=plan_budget_23_merged.drop(
    plan_budget_23_merged.columns[[22,21,16]],
    axis=1,
)

budget_23_not_merged= budget_23_only_Ids[~budget_23_only_Ids[budget_23_only_Ids.columns[15]].isin(plan_23[plan_23.columns[7]])]

plan_23_not_merged=plan_23[~plan_23[plan_23.columns[7]].isin(budget_23_only_Ids[budget_23_only_Ids.columns[15]])]

# budget_23_no_Ids
# plan_23_not_merged
# budget_23_not_merged
plan_budget_23_merged

Unnamed: 0_level_0 Unnamed: 1_level_0  \
                                   Unnamed: 0_level_1 Unnamed: 1_level_1   
0       Siltasaarenkatu 2-3 linja (Siltasaarenportti)                 K1   
1                                       Helsinginkuja                 K1   
2          Koskelan sairaalan ja pesuloiden alue (AM)               K5.1   
3                                 - Kunnalliskodintie               K5.1   
4      - Koskelantie (Käpyläntie - Kunnalliskodintie)               K5.1   
5                                        - Käpyläntie               K5.1   
6                                    - Koskelanakseli               K5.1   
7                                    - Paviljonkikuja               K5.1   
8                                  - Antti Korpin tie               K5.1   
9                                     - Kappelinkulku               K5.1   
10                                         Leanportti                 K1   
11                                         Paturintie               K5.1   
12  Kivipadontie (Maapadontie-Käskynhaltijantien r...               K5.1   
13  Pirjontien ja Pirkkolantien alue (AM/Raide-Jok...               None   
14     Maunulantie välillä Pirkkolantie-Maunulanpolku               K5.1   
15                                        Pirjonaukio               K5.1   
16                              - Lehtotielle hidaste               K5.1   
17  Metsäpurontie - Koivikkotie -Pirkkolantie + Kä...               K5.1   
18         Metsäpurontie (Pakilantie- Rajametsäntie)                K5.1   
19  Lampuotilantie itä (kääntöpaikka) + Keski jatk...               K5.1   
20          Käskynhaltijantien alue (AM/Raide-Jokeri)               None   
21                                       Maapadontie                K5.1   
22                                     Kivalterinkuja               K5.1   
23  Teinintie (sis. Kinkeripolku, ei länsiosaa lop...               K5.1   
24                                      Mestarintien                K5.1   
25               Maaherrantien alue (AM/Raide-Jokeri)               None   
26  Larin Kyöstin tie (-polku, Veräjänaukio ja -po...               K5.1   
27  Larin Kyöstin tie (-polku, Veräjänaukio ja -po...               K5.1   
28    Länsi-Pakilan liikennejärjestelyt (Bagge hanke)                 K4   
29                                    Viikinmäki (AM)               K5.1   
30                                        Harjannetie               K5.1   
31                              Ristiretkeläistenkatu               K5.1   
32  Säterintien alue: Säterintie, Säterinportti, M...               K5.1   
33         Maatullinpuiston eteläosan kaava-alue (AM)               None   
34                                 Rintamasotilaantie                 K4   
35                                           Rahkatie                 K4   
36                                         Takalantie                 K4   
37                                     Lähdeniityntie                 K4   
38                        Fallkullan kiilan alue (AM)               None   
39                                  Jokipoikasenkaari               K5.1   
40                                    Jokipoikasentie               K5.1   
41            Joutsentie (Tasankotie - Joutsenraitti)               K5.1   
42                                   Jäkäläpolku (LP)               K5.1   
43             Jäkälätie (v Jäkäläpolku - Joutsentie)               K5.1   
44                                         Sammaltori               K5.1   
45                                       Smoltinkaari               K5.1   
46                                        Smoltinkuja               K5.1   
47                                       Smoltinkulku               K5.1   
48                    Siltalanpuiston kaava-alue (AM)               K5.1   
49                                   Kiertotähdenkuja               K5.1   

   asuntotuotan. ennuste   2023   2024   2025   2026   2027   2028   2029  \
     

In [259]:
budget_23_sp3_data = [
    {
        "name": name,
        "category": category,
        "effectHousing": effectHousing,
        "costForecast": costForecast,
        "budgetProposalCurrentYearPlus1": bpCPlus1,
        "budgetProposalCurrentYearPlus2": bpCPlus2,
        "preliminaryCurrentYearPlus3": pcyPlus3,
        "preliminaryCurrentYearPlus4": pcyPlus4,
        "preliminaryCurrentYearPlus5": pcyPlus5,
        "preliminaryCurrentYearPlus6": pcyPlus6,
        "preliminaryCurrentYearPlus7": pcyPlus7,
        "preliminaryCurrentYearPlus8": pcyPlus8,
        "preliminaryCurrentYearPlus9": pcyPlus9,
        "preliminaryCurrentYearPlus10": pcyPlus10,
        "note_content": note_content,
        "hkrId": hkrId,
    }
    for name, category, effectHousing, costForecast, bpCPlus1, bpCPlus2, pcyPlus3, pcyPlus4, pcyPlus5, pcyPlus6, pcyPlus7, pcyPlus8, pcyPlus9, pcyPlus10, note_content, hkrId in zip(
        budget_23.iloc[:, 0],
        budget_23.iloc[:, 1],
        budget_23.iloc[:, 2],
        budget_23.iloc[:, 3],
        budget_23.iloc[:, 4],
        budget_23.iloc[:, 5],
        budget_23.iloc[:, 6],
        budget_23.iloc[:, 7],
        budget_23.iloc[:, 8],
        budget_23.iloc[:, 9],
        budget_23.iloc[:, 10],
        budget_23.iloc[:, 11],
        budget_23.iloc[:, 12],
        budget_23.iloc[:, 13],
        budget_23.iloc[:, 14],
        budget_23.iloc[:, 15],
    )
]

In [260]:
budget_23_sp3_data

[{'name': 'Siltasaarenkatu (Toinen linja - Kolmas linja)',
  'category': 'K1',
  'effectHousing': 'K',
  'costForecast': 1000.0,
  'budgetProposalCurrentYearPlus1': None,
  'budgetProposalCurrentYearPlus2': None,
  'preliminaryCurrentYearPlus3': None,
  'preliminaryCurrentYearPlus4': None,
  'preliminaryCurrentYearPlus5': None,
  'preliminaryCurrentYearPlus6': None,
  'preliminaryCurrentYearPlus7': None,
  'preliminaryCurrentYearPlus8': None,
  'preliminaryCurrentYearPlus9': None,
  'preliminaryCurrentYearPlus10': None,
  'note_content': '. 23 0,3 milj. Kivityös 24 0,5 milj.  TARKISTA VARAUS 24?',
  'hkrId': 2637},
 {'name': 'Hakaniementori + (Länsipää Miina Sillanpään katu)',
  'category': 'K5.1',
  'effectHousing': 'E',
  'costForecast': 3000.0,
  'budgetProposalCurrentYearPlus1': None,
  'budgetProposalCurrentYearPlus2': None,
  'preliminaryCurrentYearPlus3': None,
  'preliminaryCurrentYearPlus4': None,
  'preliminaryCurrentYearPlus5': None,
  'preliminaryCurrentYearPlus6': None,
  